# Manipulating data
In this chapter, you'll learn about the pyspark.sql module, which provides optimized data queries to your Spark session.

## Creating columns
In this chapter, you'll learn how to use the methods defined by Spark's DataFrame class to perform common data operations.

Let's look at performing column-wise operations. In Spark you can do this using the .withColumn() method, which takes two arguments. First, a string with the name of your new column, and second the new column itself.

The new column must be an object of class Column. Creating one of these is as easy as extracting a column from your DataFrame using df.colName.

Updating a Spark DataFrame is somewhat different than working in pandas because the Spark DataFrame is immutable. This means that it can't be changed, and so columns can't be updated in place.

Thus, all these methods return a new DataFrame. To overwrite the original DataFrame you must reassign the returned DataFrame using the method like so:

```python
df = df.withColumn("newCol", df.oldCol + 1)
```

The above code creates a DataFrame with the same columns as df plus a new column, newCol, where every entry is equal to the corresponding entry from oldCol, plus one.

To overwrite an existing column, just pass the name of the column as the first argument!

In [2]:
# Don't change this file path
file_path = ".../data/datacamp/flights"

# Read in the airports data
flights = spark.read.parquet(file_path)

# Show the head
flights.show(5)

# Add duration_hrs
flights = flights.withColumn("duration_hrs", flights.air_time / 60)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   11| 24|   502.0|     -8.0|   647.0|    -36.0|     UA| N36472|  1523|   SEA| SFO|    90.0|     679| 5.0|   2.0|
|2014|    4|  6|  1126.0|     11.0|  1720.0|     20.0|     AA| N3KDAA|  1311|   SEA| DFW|   205.0|    1660|11.0|  26.0|
|2014|    4| 10|   959.0|     -1.0|  1310.0|    -20.0|     WN| N7812G|   761|   PDX| DEN|   118.0|     991| 9.0|  59.0|
|2014|    1| 19|  1255.0|     -5.0|  1803.0|    -14.0|     DL| N643DL|  1950|   SEA| MSP|   154.0|    1399|12.0|  55.0|
|2014|    8|  6|  1751.0|      1.0|  2028.0|     -7.0|     WN| N7701B|   899|   SEA| SAN|   137.0|    1050|17.0|  51.0|
+----+-----+---+--------+---------+-----

## SQL in a nutshell
As you move forward, it will help to have a basic understanding of SQL. A more in depth look can be found here.

A SQL query returns a table derived from one or more tables contained in a database.

Every SQL query is made up of commands that tell the database what you want to do with the data. The two commands that every query has to contain are SELECT and FROM.

The SELECT command is followed by the columns you want in the resulting table.

The FROM command is followed by the name of the table that contains those columns. The minimal SQL query is:

```sql
SELECT * FROM my_table;
```

The * selects all columns, so this returns the entire table named my_table.

Similar to .withColumn(), you can do column-wise computations within a SELECT statement. For example,

```sql
SELECT origin, dest, air_time / 60 FROM flights;
```
returns a table with the origin, destination, and duration in hours for each flight.

Another commonly used command is WHERE. This command filters the rows of the table based on some logical condition you specify. The resulting table contains the rows where your condition is true. For example, if you had a table of students and grades you could do:

```sql
SELECT * FROM students
WHERE grade = 'A';
```
to select all the columns and the rows containing information about students who got As.



## SQL in a nutshell (2)
Another common database task is aggregation. That is, reducing your data by breaking it into chunks and summarizing each chunk.

This is done in SQL using the GROUP BY command. This command breaks your data into groups and applies a function from your SELECT statement to each group.

For example, if you wanted to count the number of flights from each of two origin destinations, you could use the query

```sql
SELECT COUNT(*) FROM flights
GROUP BY origin;
```
GROUP BY origin tells SQL that you want the output to have a row for each unique value of the origin column. The SELECT statement selects the values you want to populate each of the columns. Here, we want to COUNT() every row in each of the groups.

It's possible to GROUP BY more than one column. When you do this, the resulting table has a row for every combination of the unique values in each column. The following query counts the number of flights from SEA and PDX to every destination airport:

```sql
SELECT origin, dest, COUNT(*) FROM flights
GROUP BY origin, dest;
```
The output will have a row for every combination of the values in origin and dest (i.e. a row listing each origin and destination that a flight flew to). There will also be a column with the COUNT() of all the rows in each group.

## Filtering Data
Now that you have a bit of SQL know-how under your belt, it's easier to talk about the analogous operations using Spark DataFrames.

Let's take a look at the .filter() method. As you might suspect, this is the Spark counterpart of SQL's WHERE clause. The .filter() method takes either an expression that would follow the WHERE clause of a SQL expression as a string, or a Spark Column of boolean (True/False) values.

For example, the following two expressions will produce the same output:

```python
flights.filter("air_time > 120").show()
flights.filter(flights.air_time > 120).show()
```
Notice that in the first case, we pass a string to .filter(). In SQL, we would write this filtering task as SELECT * FROM flights WHERE air_time > 120. Spark's .filter() can accept any expression that could go in the WHEREclause of a SQL query (in this case, "air_time > 120"), as long as it is passed as a string. Notice that in this case, we do not reference the name of the table in the string -- as we wouldn't in the SQL request.

In the second case, we actually pass a column of boolean values to .filter(). Remember that flights.air_time > 120 returns a column of boolean values that has True in place of those records in flights.air_time that are over 120, and False otherwise.

In [3]:
# Filter flights by passing a string
long_flights1 = flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.distance > 1000)

# Print the data to check they're equal
long_flights1.show(5)
long_flights2.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    4|  6|  1126.0|     11.0|  1720.0|     20.0|     AA| N3KDAA|  1311|   SEA| DFW|   205.0|    1660|11.0|  26.0|3.4166666666666665|
|2014|    1| 19|  1255.0|     -5.0|  1803.0|    -14.0|     DL| N643DL|  1950|   SEA| MSP|   154.0|    1399|12.0|  55.0| 2.566666666666667|
|2014|    8|  6|  1751.0|      1.0|  2028.0|     -7.0|     WN| N7701B|   899|   SEA| SAN|   137.0|    1050|17.0|  51.0| 2.283333333333333|
|2014|    3| 21|  1638.0|     -2.0|  1953.0|     -2.0|     AS| N566AS|   867|   SEA| OGG|   353.0|    2640|16.0|  38.0| 5.883333333333334|
|2014|   10|  9|  1231.0|  

# Selecting
The Spark variant of SQL's SELECT is the .select() method. This method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a string (one for each column) or a column object (using the df.colName syntax). When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside .withColumn().

The difference between .select() and .withColumn() methods is that .select() returns only the columns you specify, while .withColumn() returns all the columns of the DataFrame in addition to the one you defined. It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use .select() and not .withColumn().

Remember, a SparkSession called spark is already in your workspace, along with the Spark DataFrame flights.

In [4]:
# Select the first set of columns
selected1 = flights.select("tailnum", "origin", "dest")

# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

## Selecting II
Similar to SQL, you can also use the .select() method to perform column-wise operations. When you're selecting a column using the df.colName notation, you can perform any column operation and the .select() method will return the transformed column. For example,

```python
flights.select(flights.air_time/60)
```

returns a column of flight durations in hours instead of minutes. You can also use the .alias() method to rename a column you're selecting. So if you wanted to .select() the column duration_hrs (which isn't in your DataFrame) you could do

```python
flights.select((flights.air_time/60).alias("duration_hrs"))
```

The equivalent Spark DataFrame method .selectExpr() takes SQL expressions as a string:

flights.selectExpr("air_time/60 as duration_hrs")
with the SQL as keyword being equivalent to the .alias() method. To select multiple columns, you can pass multiple strings.

In [5]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")

## Aggregating
All of the common aggregation methods, like .min(), .max(), and .count() are GroupedData methods. These are created by calling the .groupBy() DataFrame method. You'll learn exactly what that means in a few exercises. For now, all you have to do to use these functions is call that method on your DataFrame. For example, to find the minimum value of a column, col, in a DataFrame, df, you could do
```python
df.groupBy().min("col").show()
```
This creates a GroupedData object (so you can use the .min() method), then finds the minimum value in col, and returns it as a DataFrame.

In [6]:
# Find the shortest flight from PDX in terms of distance
flights.filter(flights.origin == "PDX").groupBy().min("distance").show(5)

# Find the longest flight from SEA in terms of air time
flights.filter(flights.origin == "SEA").groupBy().max("air_time").show(5)

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|          NaN|
+-------------+



## Aggregating II
To get you familiar with more of the built in aggregation methods, here's a few more exercises involving the flights table!

In [7]:
# Average duration of Delta flights
flights.filter(flights.carrier == "DL").filter(flights.origin == "SEA").groupBy().avg("air_time").show(5)

# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show(5)

+-------------+
|avg(air_time)|
+-------------+
|          NaN|
+-------------+

+-----------------+
|sum(duration_hrs)|
+-----------------+
|              NaN|
+-----------------+



## Grouping and Aggregating I
Part of what makes aggregating so powerful is the addition of groups. PySpark has a whole class devoted to grouped data frames: pyspark.sql.GroupedData, which you saw in the last two exercises.

You've learned how to create a grouped DataFrame by calling the .groupBy() method on a DataFrame with no arguments.

Now you'll see that when you pass the name of one or more columns in your DataFrame to the .groupBy() method, the aggregation methods behave like when you use a GROUP BY statement in a SQL query!

In [8]:
# Group by tailnum
by_plane = flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().show(5)

# Group by origin
by_origin = flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show(5)

+-------+-----+
|tailnum|count|
+-------+-----+
| N369SW|    6|
| N767SK|    1|
| N929SW|    2|
| N770SA|    5|
| N521UW|    1|
+-------+-----+
only showing top 5 rows

+------+-------------+
|origin|avg(air_time)|
+------+-------------+
|   SEA|          NaN|
|   PDX|          NaN|
+------+-------------+



## Grouping and Aggregating II
In addition to the GroupedData methods you've already seen, there is also the .agg() method. This method lets you pass an aggregate column expression that uses any of the aggregate functions from the pyspark.sql.functions submodule.

This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule take the name of a column in a GroupedData table.

In [9]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# Group by month and dest
by_month_dest = flights.groupBy("month", "dest")

# Average departure delay by month and destination
by_month_dest.avg("dep_delay").show(5)

# Standard deviation of departure delay
by_month_dest.agg(F.stddev("dep_delay")).show(5)

+-----+----+-------------------+
|month|dest|     avg(dep_delay)|
+-----+----+-------------------+
|    7| BWI|             25.625|
|    9| SJC|                1.6|
|   12| TUS| -4.666666666666667|
|    3| MCO|                7.0|
|   12| BUR|-2.8461538461538463|
+-----+----+-------------------+
only showing top 5 rows

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    7| BWI|     48.05930116607428|
|    9| SJC|    11.740293129160818|
|   12| TUS|    3.7859388972001824|
|    3| MCO|                   NaN|
|   12| BUR|     6.148796004722575|
+-----+----+----------------------+
only showing top 5 rows



## Joining
Another very common data operation is the join. Joins are a whole topic unto themselves, so in this course we'll just look at simple joins. If you'd like to learn more about joins, you can take a look here.

A join will combine two different tables along a column that they share. This column is called the key. Examples of keys here include the tailnum and carrier columns from the flights table.

For example, suppose that you want to know more information about the plane that flew a flight than just the tail number. This information isn't in the flights table because the same plane flies many different flights over the course of two years, so including this information in every row would result in a lot of duplication. To avoid this, you'd have a second table that has only one row for each plane and whose columns list all the information about the plane, including its tail number. You could call this table planes

When you join the flights table to this table of airplane information, you're adding all the columns from the planes table to the flights table. To fill these columns with information, you'll look at the tail number from the flights table and find the matching one in the planes table, and then use that row to fill out all the new columns.

Now you'll have a much bigger table than before, but now every row has all information about the plane that flew that flight!

## Joining II
In PySpark, joins are performed using the DataFrame method .join(). This method takes three arguments. The first is the second DataFrame that you want to join with the first one. The second argument, on, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. The third argument, how, specifies the kind of join to perform. In this course we'll always use the value how="leftouter".

In [12]:
from pandas import read_csv

def read_pandas_export_parquet(name, npartitions=10):
    spark.createDataFrame(read_csv(f"../data/{name}.csv")).coalesce(npartitions) \
        .write.mode('overwrite') \
        .parquet(f".../data/datacamp/{name}")

In [14]:
# Don't change this file path
file_path = ".../data/datacamp/airports"

# Read in the airports data
airports = spark.read.parquet(file_path)

# Examine the data
print(airports.show())

# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")

# Join the DataFrames
flights_with_airports = flights.join(airports, on="dest", how="leftouter")

# Examine the new DataFrame
print(flights_with_airports.show(5))

+---+--------------------+------------------+-------------------+----+---+---+
|faa|                name|               lat|                lon| alt| tz|dst|
+---+--------------------+------------------+-------------------+----+---+---+
|ROC|Greater Rochester...|         43.118866|         -77.672389| 559| -5|  A|
|ROW|Roswell Intl Air ...|         33.301556|        -104.530556|3671| -7|  A|
|RSH|Russian Mission A...|            61.775|        -161.319444|  51| -9|  A|
|RSJ|Rosario Seaplane ...|         48.645556|-122.86805600000001|   0| -8|  A|
|RST|           Rochester|43.908283000000004| -92.50001400000001|1317| -6|  A|
|RSW|Southwest Florida...|         26.536167|         -81.755167|  30| -5|  A|
|RUT|Rutland State Air...|           43.5294|           -72.9496| 787| -5|  A|
|RVS|Richard Lloyd Jon...|        36.0396111| -95.98463890000001| 638| -5|  A|
|RWI|Rocky Mount Wilso...|           35.8563|           -77.8919| 159| -5|  A|
|RWL|Rawlins Municipal...|        41.8055975|-107.19